resize selenium window?

In [ ]:

const rows = 6;
const columns = 6;
var screen;

var getScreenSize = () => {
    return client
        .windowHandle()
        .then(r => client.windowHandleMaximize(r.value))
        .catch(e => {})
        .windowHandleSize()
        .then(r => (screen = r.value))
        .catch(e => console.log(e))
}

var resizeWindow = (i = 0) => {
    i = i + (typeof OFFSET !== 'undefined' ? OFFSET : 0);
    var hwnd;
    return client
        .windowHandle()
        .then(r => client.window((hwnd = r.value)))
        .then(() => getScreenSize())
        .then(screen => {
            const newPosition = {
                x: Math.floor(i % columns)
                    * Math.floor(screen.width / columns),
                y: Math.floor(i / columns % rows)
                    * Math.floor(screen.height / rows)
            };
            console.log(newPosition);
            return client.windowHandlePosition(hwnd, newPosition)
        })
        .then(() => client.windowHandleSize(hwnd, {
            width: Math.floor(screen.width / columns),
            height: Math.floor(screen.height / rows)
        }))
        .catch(e => console.log(e))
}
if(typeof client.resizeWindow === 'undefined') {
    client.addCommand('resizeWindow', resizeWindow);
}
module.exports = resizeWindow;


only one window?



In [ ]:
var importer = require('../Core');

var closeAllTabs = (keep) => {
    return client
        .getTabIds()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.length; i++) {
                if(h[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .switchTab(h[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var closeAllWindows = (keep) => {
    return client
        .windowHandles()
        .then(h => {
            var promises = [];
            for(var i = 0; i < h.value.length; i++) {
                if(h.value[i] === keep) {
                    continue;
                }
                promises.push((i => resolve => client
                              .window(h.value[i])
                              .close()
                              .then(() => resolve())
                              .catch(e => resolve()))
                              .apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .window(keep)
        .catch(e => console.log(e))
}

var onlyOneWindow = () => {
    return client
        .windowHandle()
        .then(r => client.window(r.value))
        .newWindow('https://google.com')
        .windowHandle()
        .then(r => closeAllWindows(r.value))
        .getCurrentTabId()
        .then(r => closeAllTabs(r))
        .resizeWindow(0)
        .catch(e => console.log(e))
    // TODO: close all tabs
}
module.exports = onlyOneWindow;



tile selenium chrome windows?



In [ ]:
var importer = require('../Core');

var openUrl = (url) => {
    var handles;
    return client
        .windowHandles()
        .then(h => handles = h.value)
        .execute((url, width, height) => {
            window.open(url, '', 'width=1350,height=1024');
        }, url)
        .windowHandles()
        .then(h => h.value.filter(hwnd => handles.indexOf(hwnd) === -1)[0])
        .catch(e => console.log(e))
    /*
    // TODO: use this openSPA()
    var id = Math.random().toString(36).substring(7);
    return client
        .execute((url, id, width, height) => {
            var a = document.createElement('a');
            a.setAttribute('onclick', 'window.open("' + url + '", "", "width=' + width + ',height=' + height + '"); return false;');
            a.setAttribute('id', id);
            a.href = url;
            a.style.position = 'absolute';
            a.style.zIndex = 4294967295;
            a.style.top = 0;
            a.style.left = 0;
            a.style.bottom = 0;
            a.style.right = 0;
            document.body.appendChild(a);
            setTimeout(() => a.remove(), 200);
        }, url, id, Math.floor(screen.width / columns), Math.floor(screen.height / rows))
        .click('a[id="' + id + '"]')
        */
}

var createNewWindows = (urls) => {
    var hwnd;
    return client
        .windowHandle()
        .then(r => hwnd = r.value)
        .windowHandles()
        .then(r => {
            // -1 first window is already used by sliced url
            const count = urls.length - (r.value.length - 1);
            const promises = [];
            for(var i = 0; i < count; i++) {
                promises.push((i => resolve => {
                    console.log(urls[i]);
                    return openUrl(urls[i])
                        .window(hwnd)
                        .then(h => resolve(h))
                        .catch(e => { console.log(e); resolve() })
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .windowHandles()
        // position all windows after opening
        .then(r => {
            const promises = [];
            for(var i = 0; i < r.value.length; i++) {
                promises.push((i => resolve => {
                    return client
                        .window(r.value[i])
                        .resizeWindow(i)
                        .then(() => resolve())
                        .catch(e => {console.log(e); resolve()})
                }).apply(this, [i]));
            }
            return importer.runAllPromises(promises);
        })
        .catch(e => console.log(e))
}

var resizeWindow;
var tileWindows = (urls) => {
    return client
        .resizeWindow(0)
        .url(typeof urls === 'string' ? urls : urls[0])
        .then(h => typeof urls === 'string' ? [] : createNewWindows(urls.slice(1)))
        .windowHandles()
        .catch(e => console.log(e))
}
module.exports = tileWindows;



In [12]:
var importer = require('../Core');

var query = 'list of search engines';

$$.async();
//importer.import('child process')
    //.then(execCmd => Promise.all([
    //    execCmd('node -e "require(\'/Users/briancullinan/jupytangular2/Core\').import(\'run todays calendar events\').then(runner => runner()).then(e => process.exit(e)).catch(e=>{console.log(e); process.exit(e) });"'),
    //    execCmd('node -e "require(\'/Users/briancullinan/jupytangular2/Core\').import(\'run todays calendar events\').then(runner => runner()).then(e => process.exit(e)).catch(e=>{console.log(e); process.exit(e) });"')
    //]))
// get multiple sets of 9 working for single page scraping in parallel
importer.import('selenium cell')
    .then(runSeleniumCell => runSeleniumCell('tile chrome windows'))
    .then(tileWindows => tileWindows([
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'https://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://archive.org/search.php?query=' + query,
    ]))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



Initializing webdriver on localhost
[ '69b79e52b8036c92ec1f5850d6204f32',
  'bcd0b1a85c4702601e1c3a6ca6f0da36' ]
{ x: 0, y: 3048 }
{ x: 0, y: 3048 }
https://www.bing.com/search?q=list of search engines
https://search.yahoo.com/search?p=list of search engines
https://www.ask.com/web?q=list of search engines
https://search.aol.com/aol/search?q=list of search engines
https://www.baidu.com/s?wd=list of search engines
https://www.wolframalpha.com/input/?i=list of search engines
https://duckduckgo.com/?q=list of search engines
https://archive.org/search.php?query=list of search engines
{ Error: unknown command: session/69b79e52b8036c92ec1f5850d6204f32/window/maximize
    at windowHandleMaximize("CDwindow-3e11e743-f442-4f52-87f3-99412b24abec") - index.js:232:23
    at windowHandleSize() - orchestration.ipynb[0]:10:10
  message: 'unknown command: session/69b79e52b8036c92ec1f5850d6204f32/window/maximize',
  type: 'RuntimeError' }
{ x: 0, y: 3048 }
{ x: 1360, y: 3036 }
{ Error
  message: 'unknow

{ sessionId: '69b79e52b8036c92ec1f5850d6204f32',
  status: 0,
  value: 
   [ 'CDwindow-3e11e743-f442-4f52-87f3-99412b24abec',
     'CDwindow-8c4d03f7-d9c4-4d97-8dfc-63bb0b3b69e5',
     'CDwindow-e3150bb4-7796-4490-82ef-8a1a61bc34c6',
     'CDwindow-dcd80e1b-ac11-417a-8364-3bc7114b0a1c',
     'CDwindow-2de36e77-1ac7-4a44-96d8-ba005a74056c',
     'CDwindow-00341ed4-9991-43d4-a6fd-f6ae1bae0e11',
     'CDwindow-7b699b9f-7e9a-4a2d-bdff-9dee323c9f13',
     'CDwindow-6b165b73-6cc2-4657-8a7e-1b27b4e0708b',
     'CDwindow-9f18cd5a-96cf-4f61-a847-e3e71ab77ed3' ] }